In [0]:
dbutils.fs.ls("/FileStore/tables")

Out[34]: [FileInfo(path='dbfs:/FileStore/tables/0ed442ca38ad_user_table/', name='0ed442ca38ad_user_table/', size=0, modificationTime=1703025897786),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials-1.csv', name='authentication_credentials-1.csv', size=97, modificationTime=1702911405000),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials-2.csv', name='authentication_credentials-2.csv', size=97, modificationTime=1702911558000),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=97, modificationTime=1702911748000),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1703025897786),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1703025897786),
 FileInfo(path='dbfs:/FileStore/tables/single_json_file.json', name='single_json_file.json', size=293, modificationTime=1701857903000),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1703025897786)]

In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [0]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv") # Specify file type to be csv

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a1153066525-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/incoming"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[38]: True

In [0]:
# test it mounted
display(dbutils.fs.ls(MOUNT_NAME + "/../..")) # seems like I could put anything here!

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,1703025931048
dbfs:/Volume/,Volume/,0,0
dbfs:/Volumes/,Volumes/,0,0
dbfs:/checkpoint/,checkpoint/,0,1703025931048
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/delta/,delta/,0,1703025931048
dbfs:/df_pin.csv/,df_pin.csv/,0,1703025931048
dbfs:/df_pin.parquet/,df_pin.parquet/,0,1703025931048
dbfs:/geo_dirty.csv/,geo_dirty.csv/,0,1703025931048


In [0]:
# File location and type
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = MOUNT_NAME + "/topics/0a1153066525.pin/partition=0/*.json" 
# Read in JSONs from mounted S3 bucket
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(df_pin)

# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = MOUNT_NAME + "/topics/0a1153066525.pin/partition=0/*.json" 
# Read in JSONs from mounted S3 bucket
df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(df_geo)

# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_user = MOUNT_NAME + "/topics/0a1153066525.pin/partition=0/*.json" 
# Read in JSONs from mounted S3 bucket
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(df_pin)


category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
mens-fashion,No description available Story format,0,User Info Error,Image src error.,7528,multi-video(story page format),User Info Error,Local save in /data/mens-fashion,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",No Title Data Available,fbe53c66-3442-4773-b19e-d3ec6f54dddf


category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
mens-fashion,No description available Story format,0,User Info Error,Image src error.,7528,multi-video(story page format),User Info Error,Local save in /data/mens-fashion,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",No Title Data Available,fbe53c66-3442-4773-b19e-d3ec6f54dddf


category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
mens-fashion,No description available Story format,0,User Info Error,Image src error.,7528,multi-video(story page format),User Info Error,Local save in /data/mens-fashion,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",No Title Data Available,fbe53c66-3442-4773-b19e-d3ec6f54dddf


In [0]:
dbutils.fs.unmount(MOUNT_NAME)

/mnt/incoming has been unmounted.
Out[41]: True